In [1]:
import numpy as np

# Implementacja NSGA-II

In [3]:
class NSGAII:
    def __init__(self, chromosome_length, population_size, initial_population, initial_objective_values):
        self.chromosome_length = chromosome_length
        self.population_size = population_size
        if type(initial_population) is not np.ndarray:
            raise ValueError("initial population must be a numpy array")
        if initial_population.shape != (population_size, chromosome_length):
            raise ValueError("initial population must be of shape (population_size, chromosome_length)")
        self.current_population = initial_population
        if type(initial_objective_values) is not np.ndarray:
            raise ValueError("initial objective values must be a numpy array")
        if initial_population.shape != (population_size, chromosome_length):
            raise ValueError("initial population values must be of shape (population_size, objectives_number)")
        self.current_objective_values = initial_objective_values

    def get_population(self):
        return self.current_population

    def non_dominated_sort(self, population_values):
        def dominates(vals1, vals2):
            return np.all(vals1 >= vals2) and np.any(vals1 > vals2)
        # TODO
        # implementation of fast non dominated sort

        

    def generate_children(self, objective_values, number_of_offspring = self.population_size):
        # TODO
        # parent selection
        # recombination
        # mutation

    def replace_population(self, children_population, children_objective_values):
        # TODO
        # combine current population and children population
        # sort it using non dominated sort
        self.current_population = combined_population[:population_size]
        self.current_objective_values = combined_objective_values[:population_size]
